In [1]:
import tweepy 
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re 
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import csv



from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.regularizers import l1_l2, l2, l1


Using TensorFlow backend.


In [2]:
#twitter Account Keys
consumerKey = "L7o3Yf6HD1o55Q9qcJuuoYcuC"
consumerSecret = "s3kZeTFtd6DO5Jvkw6MkFfOwye38TqnksNEEk8uakZujVNFC2D"
accessToken = "3235982888-D8VdbzDEUmtLaQvBjMazprLLOJpr0U6I3ZKhGwl"
accessTokenSecret = "RvXhR9hLRS0C9dyHEu2VvMJdRG6qIavATJRSxZP1Hpm8x"

In [3]:
# Create the authentication object
authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret) 

In [4]:
# Set the access token and access token secret
authenticate.set_access_token(accessToken, accessTokenSecret) 

In [5]:
# Creating the API object while passing in auth information
api = tweepy.API(authenticate, wait_on_rate_limit = True)

In [6]:
bg = api.user_timeline(screen_name="BillGates", count = 200, lang ="en", tweet_mode="extended")
bo = api.user_timeline(screen_name="BarackObama", count = 200, lang ="en", tweet_mode="extended")
hc = api.user_timeline(screen_name="HillaryClinton", count = 200, lang ="en", tweet_mode="extended")
bj= api.user_timeline(screen_name="BorisJohnson", count = 200, lang ="en", tweet_mode="extended")
bc = api.user_timeline(screen_name="jeremycorbyn", count = 200, lang ="en", tweet_mode="extended")



In [7]:
# Create a dataframe with a column called Tweets
df1 = pd.DataFrame([tweet.full_text for tweet in bo], columns=['Tweets'])


df1['Writer'] = 'BarackObama'

df2 =pd.DataFrame([tweet.full_text for tweet in bg], columns=['Tweets'])

df2['Writer'] = 'BillGates'

df3 = pd.DataFrame([tweet.full_text for tweet in hc], columns=['Tweets'])

df3['Writer'] = 'HillaryClinton'

df4 = pd.DataFrame([tweet.full_text for tweet in bj], columns=['Tweets'])

df4['Writer'] = 'BorisJohnson'

df5 = pd.DataFrame([tweet.full_text for tweet in bc], columns=['Tweets'])

df5['Writer'] = 'JeremyCorbyn'

data = pd.read_csv('Dataset\Random.csv', encoding='latin-1', usecols=['SentimentText'])
data=data.astype(str)
d6=data.head(200)
d6['Writer'] = 'Random'
d6.columns=['Tweets','Writer']

df = pd.concat([df1, df2, df3, df4, df5,d6])







C:\Users\Alee\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
# Create a function to clean the tweets
import re
def cleanTxt(text):
 text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
 text = re.sub('#', '', text) # Removing '#' hash tag
 
 text = re.sub('RT[\s]+', '', text) # Removing RT
 text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
 text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
 text=text.lower() #converting to lowercase
 text = re.sub(':', '', text) # Removing  
 return text

In [9]:
# Clean the tweets
df['Tweets'] = df['Tweets'].apply(cleanTxt)

In [10]:
# Show the new dataframe with columns 'Subjectivity' & 'Polarity'
df

,Tweets,Writer
0,i will never forget the commitment to reconcil...,BarackObama
1,we have to treat voting as the most important ...,BarackObama
2,let’s guarantee that every citizen has equal r...,BarackObama
3,we can do this by making sure every american i...,BarackObama
4,we've got to fight harder to protect the right...,BarackObama
...,...,...
195,"17 told you, you would sweep haha -p",Random
196,. there r many gr8 science cartoons,Random
197,- 1 edinburgh fringe is only 10 weeks away?! i...,Random
198,why didnt you come to school?? have you done...,Random


In [11]:
#df.to_csv('Dataset/NewTweets.csv')

In [12]:
df.to_csv('Dataset/NewTweets.csv')

df.to_pickle("Dataset/NewTweets.pkl")

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(df['Tweets'])
test_vectors = vectorizer.transform(df['Tweets'])

In [14]:
import time
from sklearn import svm
from sklearn.metrics import classification_report
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, df['Writer'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(df['Writer'], prediction_linear, output_dict=True)

print('BarackObama: ', report['BarackObama'])
print('BillGates: ', report['BillGates'])
print('HillaryClinton: ', report['HillaryClinton'])
print('BorisJohnson: ', report['BorisJohnson'])
print('JeremyCorbyn: ', report['JeremyCorbyn'])
print('Random: ', report['Random'])



Training time: 0.347676s; Prediction time: 0.247769s
BarackObama:  {'precision': 0.9182692307692307, 'recall': 0.955, 'f1-score': 0.9362745098039216, 'support': 200}
BillGates:  {'precision': 0.9552238805970149, 'recall': 0.96, 'f1-score': 0.9576059850374063, 'support': 200}
HillaryClinton:  {'precision': 0.9368421052631579, 'recall': 0.89, 'f1-score': 0.9128205128205129, 'support': 200}
BorisJohnson:  {'precision': 0.9651741293532339, 'recall': 0.97, 'f1-score': 0.9675810473815462, 'support': 200}
JeremyCorbyn:  {'precision': 0.9790575916230366, 'recall': 0.935, 'f1-score': 0.9565217391304348, 'support': 200}
Random:  {'precision': 0.9090909090909091, 'recall': 0.95, 'f1-score': 0.9290953545232273, 'support': 200}


In [15]:
review = "On Memorial Day, we honor those who gave all for us. That takes different forms this year, but it’s even more vital with the loss of so many veterans to COVID-19.The way they lived, in service to one another, should be our roadmap in the months ahead."
review_vector = vectorizer.transform([review]) # vectorizing
print(classifier_linear.predict(review_vector))

['BarackObama']


In [16]:
import pickle

# pickling the vectorizer
pickle.dump(vectorizer, open('Data/vect.sav', 'wb'))
# pickling the model
pickle.dump(classifier_linear, open('Data/clas.sav', 'wb'))

In [17]:
import pickle

vectorizer = pickle.load(open("Data/vect.sav", "rb"))
classifier_linear= pickle.load(open("Data/clas.sav", "rb"))

In [18]:
review = "Class of 2020, these are not easy times. But we will get through them. And with your leadership, the world will be stronger than before." # vectorizing
review_vector = vectorizer.transform([review])
print(classifier_linear.predict(review_vector)[0])

BillGates
